In [1]:
from sklearn.metrics import roc_curve, auc
# Models
from sklearn.svm import LinearSVC #SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
# Other Packages
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.externals import joblib
import random
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

/Users/adrian/coding/rtbda_seminar/rtbda_venv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
def prepare_data (data, sample_size):
    # Create Dataframe of unique docs to use sample function from pandas with random state
    docs = pd.DataFrame(data.doc.unique())
    random_docs = docs[0].sample(n=sample_size, random_state=1).values.tolist()
    data_sample = data[data['doc'].isin(random_docs)]
    data_sample1 = data_sample.loc[:, 'doc':'chem']
    data_sample2 = data_sample.loc[:, 'word.is.lower':data.iloc[:,-1].name]

    # Replace missing values
    data_sample2 = data_sample2.fillna(data_sample2.mode().iloc[0])
    data_sample = pd.concat([data_sample1, data_sample2], axis=1, sort=False)

    # Split random_docs in test and training sets
    docs_train, docs_test = train_test_split(random_docs, random_state=1)

    # Split random_data in test and training sets
    train, test = data_sample[data_sample['doc'].isin(docs_train)], data_sample[data_sample['doc'].isin(docs_test)]
    
    return train, test

In [3]:
def n_estimators (x_train, y_train):
    n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
    train_results = []
    test_results = []

    for estimator in n_estimators:
       rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
       rf.fit(x_train, y_train)
       train_pred = rf.predict(x_train)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       train_results.append(roc_auc)
       y_pred = rf.predict(x_test)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       test_results.append(roc_auc)

    from matplotlib.legend_handler import HandlerLine2D
    line1, = plt.plot(n_estimators, train_results, 'b', label='Train AUC')
    line2, = plt.plot(n_estimators, test_results, 'r', label='Test AUC')
    plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
    plt.ylabel('AUC score')
    plt.xlabel('n_estimators')
    plt.show()

In [8]:
labels = ['chapter', 'subchapter', 'version', 'directive', 'signal', 'company', 'date','date_oldversiondate', 'date_printdate', 'date_revisiondate', 'date_validdate', 'usecase', 'usecase_con', 'usecase_pro', 'chem']

In [9]:
data = pd.read_pickle('data_model_allfeatw13_web3.pkl')

In [ ]:
train, test = prepare_data(data, 500)
colidx = data.columns.get_loc

r_paper_w = np.r_[colidx('word.is.lower'):colidx('+6_word.is.stop')]
r_own_w = np.r_[colidx('word.is.title'):colidx('+6_is.page.3')]
r_own_date_spe_w = np.r_[colidx('word.is.print.date.trigger'):colidx('+6_word.is.oldversion.date.trigger')]
r_own_date_full_w = np.r_[r_own_w, r_own_date_spe_w]
r_web_w = np.r_[colidx('0'):colidx('299')]

feat = ('Own+Paper+Wordembedding', np.r_[r_own_w, r_paper_w, r_web_w])
# Separate random_data training/test split in features and labels
x_train = train.iloc[:, feat[1]]
x_test = test.iloc[:, feat[1]]

In [ ]:
for l in labels:
    y_train = train.loc[:, l]
    y_test = test.loc[:, l]
    
    print ('************' + l + '************')
    n_estimators(x_train, y_train)
    